In [1]:
import json
from virtual_agents import *
from utils import *
from openai import OpenAI

In [ ]:
client = OpenAI(api_key="")

In [3]:
agents_prompt1 = describe_agents(research_team)
meeting_types = ["individual_meeting", "group_meeting", "working_session", "draft_writing_session", "peer_review_process"]
hypothesis = "to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods"
global_project_agenda  = f'You are working on a research project {hypothesis}.' + agents_prompt1

In [4]:
# database connection
host='localhost'
user='root'
passwd='Aged_madrid.db1'
database_name='aged_madrid'
excluded_vars = [("pacientes","motivo_baja"),("pacientes","fecha_baja"),("pacientes","fecha_final_seguimiento")]
folder_path = r"C:\Users\Lucia Carrasco\OneDrive - FUNDACION IMDEA-ALIMENTACION\Escritorio\LUCIA\AGED_MADRID\AGENTS4SCIENCE".replace("\\","/")

db_dict_path = os.path.join(folder_path, "AgedMadrid_dictdb.txt")
aged_madrid_dictdb = make_dictdb(host, user, passwd, database_name, db_dict_path, excluded_vars)

{'antropometricas': ['id', 'IMC_count_15_19', 'IMC_min_fecha_15_19', 'IMC_max_fecha_15_19', 'IMC_1a_deter_15_19', 'IMC_ult_deter_15_19', 'IMC_media_15_19', 'IMC_sd_15_19', 'IMC_cv_15_19', 'IMC_tiempo_15_19', 'talla_count_15_19', 'talla_min_fecha_15_19', 'talla_max_fecha_15_19', 'talla_1a_deter_15_19', 'talla_ult_deter_15_19', 'talla_media_15_19', 'talla_sd_15_19', 'talla_cv_15_19', 'talla_tiempo_15_19', 'peso_count_15_19', 'peso_min_fecha_15_19', 'peso_max_fecha_15_19', 'peso_1a_deter_15_19', 'peso_ult_deter_15_19', 'peso_media_15_19', 'peso_sd_15_19', 'peso_cv_15_19', 'peso_tiempo_15_19', 'perimetro_abdominal_count_15_19', 'perimetro_abdominal_min_fecha_15_19', 'perimetro_abdominal_max_fecha_15_19', 'perimetro_abdominal_1a_deter_15_19', 'perimetro_abdominal_ult_deter_15_19', 'perimetro_abdominal_media_15_19', 'perimetro_abdominal_sd_15_19', 'perimetro_abdominal_cv_15_19', 'perimetro_abdominal_tiempo_15_19', 'indice_cintura_cadera_count_15_19', 'indice_cintura_cadera_min_fecha_15_19', 

In [5]:
agent_to_assistant = {
            agent.title: client.beta.assistants.create(
            name=agent.title,
            instructions=agent.prompt,
            model=agent.model,
            temperature=0,
        )
        for agent in agents}

In [6]:
print(agent_to_assistant)

{'Primary Healthcare Professional & Coordinator': Assistant(id='asst_D7u62aVTvTSumc0f0LqZcr7V', created_at=1757754555, description=None, instructions='You are a Primary Healthcare Professional & Coordinator. Your expertise is in primary care of patients (multimorbility manage, polypharmacy and fragility in elderly populations, interpretation of clinical trajectories and identification of subtle changes and patterns that may not be apparent in aggregated data), early identification and management of risk factors (e.g., glycemic control, infection prevention), participation and coordination in translational research projects, and clinical validation of findings. Your goal is to Providing a clinical and coordinating perspective to guide the interpretation of results and their translation into practical healthcare insights. It include Model validation to ensure analytical outputs are clinically meaningful, translation of findings into actionable hypotheses or recommendations for primary ca

In [7]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [8]:
show_json(agent_to_assistant[coordinator.title])

{'id': 'asst_D7u62aVTvTSumc0f0LqZcr7V',
 'created_at': 1757754555,
 'description': None,
 'instructions': 'You are a Primary Healthcare Professional & Coordinator. Your expertise is in primary care of patients (multimorbility manage, polypharmacy and fragility in elderly populations, interpretation of clinical trajectories and identification of subtle changes and patterns that may not be apparent in aggregated data), early identification and management of risk factors (e.g., glycemic control, infection prevention), participation and coordination in translational research projects, and clinical validation of findings. Your goal is to Providing a clinical and coordinating perspective to guide the interpretation of results and their translation into practical healthcare insights. It include Model validation to ensure analytical outputs are clinically meaningful, translation of findings into actionable hypotheses or recommendations for primary care, and leadership in coordinating multidisc

## Discussion 1: Workflow design

In [9]:
interventors_d1 = {key.title: agent_to_assistant[key.title] for key in [coordinator]}
print(interventors_d1)

{'Primary Healthcare Professional & Coordinator': Assistant(id='asst_D7u62aVTvTSumc0f0LqZcr7V', created_at=1757754555, description=None, instructions='You are a Primary Healthcare Professional & Coordinator. Your expertise is in primary care of patients (multimorbility manage, polypharmacy and fragility in elderly populations, interpretation of clinical trajectories and identification of subtle changes and patterns that may not be apparent in aggregated data), early identification and management of risk factors (e.g., glycemic control, infection prevention), participation and coordination in translational research projects, and clinical validation of findings. Your goal is to Providing a clinical and coordinating perspective to guide the interpretation of results and their translation into practical healthcare insights. It include Model validation to ensure analytical outputs are clinically meaningful, translation of findings into actionable hypotheses or recommendations for primary ca

In [10]:
prompt_discussion_1 = Discussion(
    title = "d1: Workflow design",
    type = "individual meeting",
    goal = "define a provisional workflow to pursue the aim of the project",
    moderator = coordinator,
    contributors = interventors_d1, #aquí (y en individual meetings en general, quizas es más interesante meter directamente el prompt del agente) #recuerda que en este primer prompt se describe al equipo también para asignar las tareas del workflow
    materials = ("electronic health records from 500,000 adults aged over 75 in Madrid. This database originates from a retrospective longitudinal study, covering medical variables recorded during the periods 2015–2019 and 2020–2022.", aged_madrid_dictdb),
    agenda = global_project_agenda,
    request = " outline a provisional workflow to address the hypothesis defined as the project objective. The steps should be ordered and each one assigned to the agent best suited to carry it out. You should return just the provisional workflow, without introductory or closing phrases and please respond in a direct manner without emoticons. Only use a list if necessary, and if so, use a simple list.",
    deliverable = "provisional workflow")

In [11]:
print(prompt_discussion_1.prompt)

This is the beginning of a individual meeting to define a provisional workflow to pursue the aim of the project.
You are working on a research project to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods.Your research team is composed of the following members:
A Consultant in Geriatric Medicine, expert in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intrinsic parametric variation from confounding factors and comorbidities related to aging. Their goal is Providing a geriatric clinical perspective to ensure results are relevant and applicable to older patients. Their role in the project will be Offer expertise on geriatric care and validate the clinical relevance of findings, interpreting res

In [12]:
workflow, log1, thread1 = run_discussion(client, prompt_discussion_1, coordinator, interventors_d1)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_D7u62aVTvTSumc0f0LqZcr7V, content=['1. Data Extraction and Preprocessing: Clinical Data Scientist\n   - Extract relevant data from the electronic health records database.\n   - Preprocess data to ensure quality and integrity, focusing on variables related to glycemic variability, mortality, and COVID-19 status.\n\n2. Definition and Measurement of Glycemic Variability: Endocrinologist\n   - Define and validate metrics for glycemic variability using glucose and HBa1C data.\n   - Ensure clinical relevance and biological plausibility of glycemic variability measures.\n\n3. Identification of Confounding Factors: Clinical Pharmacology Specialist\n   - Identify and evaluate pharmacological factors and polypharmacy that may influence outcomes.\n   - Collaborate with the Geriatric Medicine Consultant to assess the impact of medications on mortality.\n\n4. Clinical Relevance and Validation: Consultant in Geriatric Medicine\n   - Va

In [13]:
provisional_workflow = "This is the provisional workflow outlined by Primary Healthcare Professional & Coordinator to address the main objective of the project:\n\n" + workflow

In [14]:
print(provisional_workflow)

This is the provisional workflow outlined by Primary Healthcare Professional & Coordinator to address the main objective of the project:

1. Data Extraction and Preprocessing: Clinical Data Scientist
   - Extract relevant data from the electronic health records database.
   - Preprocess data to ensure quality and integrity, focusing on variables related to glycemic variability, mortality, and COVID-19 status.

2. Definition and Measurement of Glycemic Variability: Endocrinologist
   - Define and validate metrics for glycemic variability using glucose and HBa1C data.
   - Ensure clinical relevance and biological plausibility of glycemic variability measures.

3. Identification of Confounding Factors: Clinical Pharmacology Specialist
   - Identify and evaluate pharmacological factors and polypharmacy that may influence outcomes.
   - Collaborate with the Geriatric Medicine Consultant to assess the impact of medications on mortality.

4. Clinical Relevance and Validation: Consultant in Ge

## Discussion 2: Workflow refination

In [197]:
import importlib
import utils

importlib.reload(utils)
from utils import Discussion  # reimportas lo nuevo
from utils import run_discussion  # reimportas lo nuevo



In [24]:
participants_d2 = {key.title: agent_to_assistant[key.title] for key in [*research_team, scientific_critic, coordinator]}
print(participants_d2)

{'Consultant in Geriatric Medicine': Assistant(id='asst_GuMsWZg2W78oHGAzLzUypWFU', created_at=1757754556, description=None, instructions='You are a Consultant in Geriatric Medicine. Your expertise is in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intrinsic parametric variation from confounding factors and comorbidities related to aging. Your goal is to Providing a geriatric clinical perspective to ensure results are relevant and applicable to older patients. Your role is to Offer expertise on geriatric care and validate the clinical relevance of findings, interpreting results considering age-related factors and comorbidities.', metadata={}, model='gpt-4o', name='Consultant in Geriatric Medicine', object='assistant', tools=[], response_format='auto', temperature=0.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None), 'Clinical Pharmac

In [25]:
prompt_discussion_2 = Discussion(
    title = "Workflow refination",
    moderator = coordinator,
    type = "group meeting",
    goal = "outline a final workflow to address the hypothesis defined as the project objective.",
    contributors = research_team, #aquí (y en individual meetings en general, quizas es más interesante meter directamente el prompt del agente) #recuerda que en este primer prompt se describe al equipo también para asignar las tareas del workflow
    scheme = (2, participants_d2),
    materials = ("electronic health records from 500,000 adults aged over 75 in Madrid. This database originates from a retrospective longitudinal study, covering medical variables recorded during the periods 2015–2019 and 2020–2022.", aged_madrid_dictdb),
    agenda = global_project_agenda + provisional_workflow,
    request  = " provide your initial thoughts on the agenda, pose follow-up questions to gather additional information and feedback on how to better address the agenda as well as any questions you may have to guide the team discussion.",
    participants_request="thoughts on the discussion",
    final_request = "outline the final_workflow, integrating the group’s insights to address the hypothesis defined as the project objective. The steps should be ordered and each one assigned to the agent best suited to carry it out",
    deliverable = "final workflow")

In [26]:
print(prompt_discussion_2.prompt)

This is the beginning of a group meeting to outline a final workflow to address the hypothesis defined as the project objective..
You are working on a research project to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods.Your research team is composed of the following members:
A Consultant in Geriatric Medicine, expert in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intrinsic parametric variation from confounding factors and comorbidities related to aging. Their goal is Providing a geriatric clinical perspective to ensure results are relevant and applicable to older patients. Their role in the project will be Offer expertise on geriatric care and validate the clinical relevance of findings,

In [27]:
final_workflow, log2, thread2 = run_discussion(client, prompt_discussion_2, coordinator, participants_d2)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_D7u62aVTvTSumc0f0LqZcr7V, content=["As the Primary Healthcare Professional & Coordinator, my initial thoughts on the agenda are to ensure that each team member's expertise is effectively utilized to address the hypothesis that glycemic variability (GV) is an independent predictor of all-cause mortality in patients aged 75 years or older, with and without COVID-19, during the first year of the pandemic. Here are my initial thoughts and follow-up questions to guide the team discussion:\n\n### Initial Thoughts:\n\n1. **Data Extraction and Preprocessing:**\n   - Ensure that the Clinical Data Scientist has a clear understanding of the data dictionary and the specific variables needed for analysis, particularly those related to glycemic variability, mortality, and COVID-19 status.\n   - Emphasize the importance of data quality and integrity, as this will form the foundation for all subsequent analyses.\n\n2. **Definition and Me

In [28]:
print(final_workflow)

1. **Data Extraction and Preprocessing:**
   - **Agent:** Clinical Data Scientist
   - Extract relevant data from the electronic health records database, focusing on variables related to glycemic variability (GV), mortality, and COVID-19 status.
   - Implement data cleaning procedures to address missing values, outliers, and inconsistencies, ensuring data quality and integrity.
   - Structure temporal data to capture changes over time, particularly for pre-pandemic and pandemic periods.

2. **Definition and Measurement of Glycemic Variability:**
   - **Agent:** Endocrinologist
   - Define and validate metrics for GV using glucose and HBa1C data, ensuring clinical relevance and biological plausibility.
   - Conduct sensitivity analyses to test the robustness of GV metrics across different subgroups and time periods.

3. **Identification of Confounding Factors:**
   - **Agent:** Clinical Pharmacology Specialist
   - Identify and evaluate pharmacological factors and polypharmacy that may 

In [29]:
definitive_workflow = "This is the final workflow outlined by Primary Healthcare Professional & Coordinator, including group's insights to address the main objective of the project:\n\n" + final_workflow

## Discussion 3: Define a methodology

In [30]:
developers = [epidemiologist, clinical_data_scientist]
participants_d3 = {key.title: agent_to_assistant[key.title] for key in [*developers, coordinator]}
print(participants_d3)

{'Epidemiologist & Biostatistician': Assistant(id='asst_lgEgWKKFVjAT5xkYgmGCG9zU', created_at=1757754557, description=None, instructions='You are a Epidemiologist & Biostatistician. Your expertise is in population health, statistical modeling and analysis methods in biomedical research. Your goal is to understand population-level factors affecting clinical data and apply statistical techniques to evaluate data quality and utility. Your role is to analyze demographic and epidemiological factors contributing to distribution shifts and provide statistical insights into data valuation methodologies.', metadata={}, model='gpt-4o', name='Epidemiologist & Biostatistician', object='assistant', tools=[], response_format='auto', temperature=0.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None), 'Clinical Data Scientist': Assistant(id='asst_CPQ0x1pfvDrRDyJPjfOVUlFK', created_at=1757754558, description=None, instructions='You are a Clinical D

In [31]:
prompt_discussion_3 = Discussion(
    title = "Procedure, methodology and model selection",
    moderator = coordinator,
    type = "group meeting",
    goal = "define a clear following procedure, methodology - including since variables selection, data preprocessing of raw data to the selection of models - and tools necessary to obtain the results",
    contributors = developers,
    scheme = (2, participants_d3), #aquí (y en individual meetings en general, quizas es más interesante meter directamente el prompt del agente) #recuerda que en este primer prompt se describe al equipo también para asignar las tareas del workflow
    materials = ("electronic health records from 500,000 adults aged over 75 in Madrid. This database originates from a retrospective longitudinal study, covering medical variables recorded during the periods 2015–2019 and 2020–2022.", aged_madrid_dictdb),
    agenda = global_project_agenda + "\n" + definitive_workflow,
    final_request = "please, based on the discussion so far, synthesize the key points regarding methodology, variable selection, data preprocessing, and model considerations",
    deliverable = "agreed-upon methodology, operational details, and variable selections",
    participants_request = "thoughts on the discussion",
    request = f"{coordinator.title} convene the meeting and outline the clinical priorities and expectations from the methodology discussion")

In [32]:
print(prompt_discussion_3.prompt)

This is the beginning of a group meeting to define a clear following procedure, methodology - including since variables selection, data preprocessing of raw data to the selection of models - and tools necessary to obtain the results.
You are working on a research project to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods.Your research team is composed of the following members:
A Consultant in Geriatric Medicine, expert in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intrinsic parametric variation from confounding factors and comorbidities related to aging. Their goal is Providing a geriatric clinical perspective to ensure results are relevant and applicable to older patients. Their role i

In [33]:
methodology, log3, thread3 = run_discussion(client, prompt_discussion_3, coordinator, participants_d3)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_D7u62aVTvTSumc0f0LqZcr7V, content=["**Meeting Convening and Outline of Clinical Priorities**\n\nWelcome, everyone. As we embark on this important research project, our goal is to evaluate whether glycemic variability (GV) is an independent predictor of all-cause mortality in patients aged 75 years or older, with and without COVID-19, during the first year of the pandemic. This study is crucial for understanding the impact of GV on mortality in older adults, particularly in the context of the COVID-19 pandemic.\n\n**Clinical Priorities and Expectations:**\n\n1. **Clinical Relevance:**\n   - Our primary focus is to ensure that the findings of this study are clinically meaningful and applicable to the care of older adults. We must consider the unique physiological and pathological characteristics of this population, including multimorbidity, polypharmacy, and age-related changes.\n\n2. **Integration of Multidisciplinary Insi

In [34]:
print(methodology)

Data Extraction and Preprocessing:
- Extract relevant data from EHR, focusing on glycemic variability (GV), mortality, and COVID-19 status.
- Address missing values, outliers, and inconsistencies using imputation methods and criteria for outlier handling.
- Structure temporal data to capture changes over time, aligning data points to a common timeline for pre-pandemic and pandemic periods.
- Implement data validation steps, including cross-checking against clinical benchmarks and using data visualization to identify anomalies.

Variable Selection:
- Focus on glucose and HBa1C metrics for GV, demographic variables (age, sex), comorbidities, and medication use.
- Consider potential confounders and create composite variables to account for interactions between GV and clinical factors.

Feature Engineering:
- Calculate variability metrics such as coefficients of variation and standard deviations for GV.
- Explore interaction terms and use machine learning techniques like decision trees to 

In [35]:
methodology_intro = "This is the methodology defined in the last meeting:\n" + methodology 

## Discussion 4: Working Session 

#### 4a. Split the pipeline into tasks or scripts, each assigned to a responsible agent

In [40]:
developers = [epidemiologist, clinical_data_scientist]
participants_d4a = {key.title: agent_to_assistant[key.title] for key in developers}
print(participants_d4a)

{'Epidemiologist & Biostatistician': Assistant(id='asst_lgEgWKKFVjAT5xkYgmGCG9zU', created_at=1757754557, description=None, instructions='You are a Epidemiologist & Biostatistician. Your expertise is in population health, statistical modeling and analysis methods in biomedical research. Your goal is to understand population-level factors affecting clinical data and apply statistical techniques to evaluate data quality and utility. Your role is to analyze demographic and epidemiological factors contributing to distribution shifts and provide statistical insights into data valuation methodologies.', metadata={}, model='gpt-4o', name='Epidemiologist & Biostatistician', object='assistant', tools=[], response_format='auto', temperature=0.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None), 'Clinical Data Scientist': Assistant(id='asst_CPQ0x1pfvDrRDyJPjfOVUlFK', created_at=1757754558, description=None, instructions='You are a Clinical D

In [44]:
prompt_discussion_4a = Discussion(
    title = "Code development",
    moderator = epidemiologist,
    type = "working session",
    goal = "design a clear, script-based analysis pipeline, starting from raw EHR data and ending with the results of statistical models, purposing ready-to-run scripts that compute the results of your study",
    deliverable="list of scripts names with python or R extension - according to the programming language choosen for each script- hat will be developed",
    contributors = developers,
    scheme = (2, participants_d4a), #aquí (y en individual meetings en general, quizas es más interesante meter directamente el prompt del agente) #recuerda que en este primer prompt se describe al equipo también para asignar las tareas del workflow
    materials = ("electronic health records from 500,000 adults aged over 75 in Madrid. This database originates from a retrospective longitudinal study, covering medical variables recorded during the periods 2015–2019 and 2020–2022.\nCSV files have been generated for each table from the database. Please consider using these files instead of connecting directly to the MySQL database", aged_madrid_dictdb),
    agenda = global_project_agenda + methodology_intro,
    participants_request= "discussion on how to break down and distribute the initial workflow for building the script pipeline, the programming language to use and how to assign responsibilities. Note that each script must be assigned to a single author (either the Epidemiologist & biostatician or the Clinical Data Scientist) and cannot be developed collaboratively",
    final_request= "return the list of python or R scripts names in order of execution that will be developed to let user obtain the results to achieve the objective just executing the codes on the raw data from the database. You should return a list of python tuples containing the values (script name, responsible agent).Keep in mind that each script must have exactly one responsible author, chosen exclusively from the Epidemiologist & biostatician or the Clinical Data Scientist. No collaborative or multiple authorship is allowed.",
    request  =  (
    f" convene the collaborative working session. "
    f"Keep in mind the objective: make decisions about the steps to follow, the script files to produce, and who will handle each part. Note that each script must be assigned to a single author (either the Epidemiologist & biostatician or the Clinical Data Scientist) and cannot be developed collaboratively."
    )
)

In [45]:
print(prompt_discussion_4a.prompt)

This is the beginning of a working session to design a clear, script-based analysis pipeline, starting from raw EHR data and ending with the results of statistical models, purposing ready-to-run scripts that compute the results of your study.
You are working on a research project to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods.Your research team is composed of the following members:
A Consultant in Geriatric Medicine, expert in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intrinsic parametric variation from confounding factors and comorbidities related to aging. Their goal is Providing a geriatric clinical perspective to ensure results are relevant and applicable to older patients. The

In [46]:
scripts_list, log4a, thread4 = run_discussion(client, prompt_discussion_4a, epidemiologist, participants_d4a)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_lgEgWKKFVjAT5xkYgmGCG9zU, content=["### Convening the Collaborative Working Session\n\nWelcome, team. Our objective today is to design a clear, script-based analysis pipeline for our research project. We will start from raw EHR data and end with the results of statistical models, focusing on whether glycemic variability (GV) is an independent predictor of all-cause mortality in patients aged 75 years or older, with and without COVID-19, during the first year of the pandemic.\n\n#### Round 1: Initial Perspectives\n\n**Epidemiologist & Biostatistician (Myself):**\n\n1. **Data Extraction and Preprocessing:**\n   - **Objective:** Extract relevant data focusing on GV, mortality, and COVID-19 status.\n   - **Approach:** Use Python for data extraction and preprocessing. Implement imputation methods for missing values and criteria for outlier handling. Structure temporal data to align with pre-pandemic and pandemic periods.\n   -

In [47]:
print(scripts_list)
import ast
import re
clean_text = scripts_list.replace("```", "").replace("python", "").strip().rstrip(";")
script_list = ast.literal_eval(clean_text)
print(script_list)


```python
[
    ("data_extraction_preprocessing.py", "Clinical Data Scientist"),
    ("variable_selection.R", "Epidemiologist & Biostatistician"),
    ("feature_engineering.py", "Clinical Data Scientist"),
    ("cox_modeling.R", "Epidemiologist & Biostatistician"),
    ("model_validation.py", "Clinical Data Scientist"),
    ("temporal_analysis.py", "Clinical Data Scientist"),
    ("integration_collaboration.py", "Epidemiologist & Biostatistician")
]
```
[('data_extraction_preprocessing.py', 'Clinical Data Scientist'), ('variable_selection.R', 'Epidemiologist & Biostatistician'), ('feature_engineering.py', 'Clinical Data Scientist'), ('cox_modeling.R', 'Epidemiologist & Biostatistician'), ('model_validation.py', 'Clinical Data Scientist'), ('temporal_analysis.py', 'Clinical Data Scientist'), ('integration_collaboration.py', 'Epidemiologist & Biostatistician')]


In [48]:
print(thread4)

thread_G1BY4ouM2etSiMFOsf7R59Zr


#### 4b. Formulate questions to the experts to solve doubts raised in variable selection

In [50]:
participants_d4b = {key.title: agent_to_assistant[key.title] for key in [epidemiologist, pharmacology_specialist, endocrinologist]}

In [ ]:
prompt_discussion_4b = Discussion(
    title = "Code development 2",
    moderator = epidemiologist,
    type = "working session",
    goal = "solve questions of variables metrics and develop the scripts to obtain the results",
    deliverable="a summary of decisions regarding variable metrics, which variables to prioritize, and other relevant insights to consider before starting the code development phase",
    contributors = research_team, 
    scheme = (1,participants_d4b), #aquí (y en individual meetings en general, quizas es más interesante meter directamente el prompt del agente) #recuerda que en este primer prompt se describe al equipo también para asignar las tareas del workflow
    participants_request= f"solve the questions asked by the {epidemiologist.title} to complete de code development",
    final_request= "return a summary of the decissions maked regarding variable metrics, which variables to prioritize, and other relevant insights to consider before starting the code development phase. ",
    request  = f", formulate the necessary questions based on the doubts raised in the meeting in order to achieve the objective"
)

In [52]:
print(prompt_discussion_4b.prompt)

This is the beginning of a working session to solve questions of variables metrics and develop the scripts to obtain the results.
Epidemiologist & Biostatistician will convene the meeting.Throughout the working session, participants will share their perspectives and contributions in the following order: Consultant in Geriatric Medicine, Clinical Pharmacology Specialist, Endocrinologist, Epidemiologist & Biostatistician, Clinical Data Scientist, over 1 rounds. In the subsequent rounds, the participants will incorporate their colleagues’ comments to refine and reformulate their ideas. Once all team members have given their input, the Epidemiologist & Biostatistician will consolidate all ideas in detail for future reference, to provide notes for the draft, and to serve as guidance for the developers
Please, Epidemiologist & Biostatistician, formulate the necessary questions based on the doubts raised in the meeting in order to achieve the objective


In [53]:
solved_questions, log4b, thread4b = run_discussion(client, prompt_discussion_4b, epidemiologist, participants_d4b, thread_id=thread4)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_lgEgWKKFVjAT5xkYgmGCG9zU, content=["### Convening the Working Session\n\nWelcome, team. Our goal today is to address questions related to variable metrics and develop scripts to obtain the results for our study on glycemic variability (GV) as an independent predictor of all-cause mortality in elderly patients during the COVID-19 pandemic. Let's proceed with the session.\n\n#### Questions for Discussion\n\n1. **Consultant in Geriatric Medicine:**\n   - How should we account for age-related factors and comorbidities when interpreting GV metrics in the elderly population?\n   - Are there specific clinical benchmarks or thresholds for GV that are particularly relevant for geriatric patients?\n\n2. **Clinical Pharmacology Specialist:**\n   - What pharmacological factors should we consider as potential confounders in our analysis of GV and mortality?\n   - How can we best account for polypharmacy and its impact on GV in our sta

In [54]:
print(solved_questions)

- **Variable Metrics for Glycemic Variability (GV):**
  - Prioritize the coefficient of variation (CV), standard deviation (SD), and mean amplitude of glycemic excursions (MAGE) as key metrics for measuring GV.
  - Calculate these metrics using both fasting and postprandial glucose levels, as well as HbA1c variability over time.

- **Pharmacological Factors:**
  - Include medications affecting glucose metabolism (e.g., insulin, sulfonylureas) and those indirectly influencing GV (e.g., corticosteroids, beta-blockers) as potential confounders.
  - Consider a polypharmacy index or medication count to account for the impact of multiple medications.

- **Interaction and Stratification:**
  - Explore interaction terms between GV and specific medication classes to assess their modifying effects on the relationship between GV and mortality.
  - Stratify analysis by key medication groups to evaluate differential impacts on GV and mortality outcomes.

- **Clinical Context for GV Interpretation:*

In [68]:
scripts_to_develop = [*script_list, ("requeriments_python.txt", "Clinical Data Scientist"), ("requeriments_r.txt", "Epidemiologist & Biostatistician") , ("README.md", "Clinical Data Scientist"), ("config.yaml", "Clinical Data Scientist")]
for script in scripts_to_develop:
    print(script)

('data_extraction_preprocessing.py', 'Clinical Data Scientist')
('variable_selection.R', 'Epidemiologist & Biostatistician')
('feature_engineering.py', 'Clinical Data Scientist')
('cox_modeling.R', 'Epidemiologist & Biostatistician')
('model_validation.py', 'Clinical Data Scientist')
('temporal_analysis.py', 'Clinical Data Scientist')
('integration_collaboration.py', 'Epidemiologist & Biostatistician')
('requeriments_python.txt', 'Clinical Data Scientist')
('requeriments_r.txt', 'Epidemiologist & Biostatistician')
('README.md', 'Clinical Data Scientist')
('config.yaml', 'Clinical Data Scientist')


## Discussion 5: Code development

In [ ]:
prompt_discussion_5 = Code_development(
    title = "Code development -5",
    moderator = epidemiologist,
    type = "working session ",
    goal = "provide a clear, script-based analysis pipeline, starting from raw EHR data and ending with ready-to-run scripts that compute the results of your study. Please assume that the database tables have been exported as CSV files, with each file named after its corresponding table, avoiding a direct connection to the database. You should save the data and results in directories ",
    deliverable= scripts_to_develop,
    contributors = developers,
    materials = ("electronic health records from 500,000 adults aged over 75 in Madrid. This database originates from a retrospective longitudinal study, covering medical variables recorded during the periods 2015–2019 and 2020–2022.\nCSV files have been generated for each table from the database. Please consider using these files instead of connecting directly to the MySQL database", aged_madrid_dictdb),
    agenda = global_project_agenda + methodology_intro + "\nBased on the resolution of questions, the described methodology, and the list of scripts to be generated, the objective is to produce the scripts that will process the raw database data and generate results in tables and figures necessary to draw conclusions." + f"This is the script-list (pipeline) to develop in this session: {scripts_to_develop}" + f"\nThese are the doubts solved by the research team in the last meeting:{solved_questions}\n",
    participants_request= ", please proceed to generate the script: ",
    final_request= " return a summary of the working session, including important details to consider it in manusctipt drafting. ",
    request  = "convene the collaborative working session.",
    participants_request_instructions=". Please, return just the code to execute in text format."
)
print(prompt_discussion_5.prompt)

This is the beginning of the code-development phase of the working session to provide a clear, script-based analysis pipeline, starting from raw EHR data and ending with ready-to-run scripts that compute the results of your study. Please assume that the database tables have been exported as CSV files, with each file named after its corresponding table, avoiding a direct connection to the database. You should save the data and results in directories .
You are working on a research project to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods.Your research team is composed of the following members:
A Consultant in Geriatric Medicine, expert in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intri

In [83]:
summary_ws, scripts, thread5 = run_code_development(client, prompt_discussion_5, epidemiologist, participants_d4a)

Epidemiologist & Biostatistician: Welcome, everyone, to our collaborative working session. As the Epidemiologist & Biostatistician, I'll be guiding us through the process of developing our analysis pipeline. Our goal is to create a series of scripts that will take us from raw EHR data to the final results of our study on glycemic variability (GV) as an independent predictor of all-cause mortality in elderly patients during the COVID-19 pandemic.

### Overview of the Session

1. **Data Extraction and Preprocessing**: 
   - **Objective**: Extract relevant data from the CSV files, handle missing values, outliers, and structure the data for analysis.
   - **Lead**: Clinical Data Scientist
   - **Script**: `data_extraction_preprocessing.py`

2. **Variable Selection**:
   - **Objective**: Identify key variables for analysis, focusing on GV metrics, demographic variables, comorbidities, and medication use.
   - **Lead**: Epidemiologist & Biostatistician
   - **Script**: `variable_selection.R`

### Recover scripts files

In [84]:
print(scripts)

[('Epidemiologist & Biostatistician', "Welcome, everyone, to our collaborative working session. As the Epidemiologist & Biostatistician, I'll be guiding us through the process of developing our analysis pipeline. Our goal is to create a series of scripts that will take us from raw EHR data to the final results of our study on glycemic variability (GV) as an independent predictor of all-cause mortality in elderly patients during the COVID-19 pandemic.\n\n### Overview of the Session\n\n1. **Data Extraction and Preprocessing**: \n   - **Objective**: Extract relevant data from the CSV files, handle missing values, and structure the data for analysis.\n   - **Lead**: Clinical Data Scientist\n   - **Script**: `data_extraction_preprocessing.py`\n\n2. **Variable Selection**:\n   - **Objective**: Identify key variables for analysis, focusing on GV metrics, demographic data, comorbidities, and medication use.\n   - **Lead**: Epidemiologist & Biostatistician\n   - **Script**: `variable_selection.

In [87]:
import os
scripts_names = [x for (x, y) in scripts_to_develop]
scripts_dict = dict()
print(scripts_names)
print(len(scripts_names))
for name, content, ts in scripts:
    scripts_dict[name] = content
path = r"c:\Users\Lucia Carrasco\OneDrive - FUNDACION IMDEA-ALIMENTACION\Escritorio\LUCIA\AGED_MADRID\AGENTS4SCIENCE\ChatGPT-api\scripts".replace("\\","/")
os.makedirs(path, exist_ok=True)
for key, value in scripts_dict.items():
    filename = os.path.join(path,key)
    with open(filename, "w+") as script:
        script.write(value)
print(scripts_dict)

['data_extraction_preprocessing.py', 'variable_selection.R', 'feature_engineering.py', 'cox_modeling.R', 'model_validation.py', 'temporal_analysis.py', 'integration_collaboration.py', 'requeriments_python.txt', 'requeriments_r.txt', 'README.md', 'config.yaml']
11
{'Epidemiologist & Biostatistician': '- Data Extraction and Preprocessing: Extracted relevant data from CSV files, focusing on glycemic variability (GV), mortality, and COVID-19 status. Addressed missing values using median imputation for numerical columns and mode for categorical columns. Handled outliers using the IQR method. Structured temporal data to align with pre-pandemic and pandemic periods.\n\n- Variable Selection: Selected key variables for analysis, including GV metrics (glucemia and HBa1C), demographic variables (age, sex), comorbidities, and medication use. Considered potential confounders and created composite variables to account for interactions between GV and clinical factors.\n\n- Feature Engineering: Calcul

#### Compile the results

In [104]:
results = '''
# Comprehensive Analysis Report

## Cox Proportional Hazards Model Summary
Call:
coxph(formula = final_formula, data = data)

  n= 530229, number of events= 530229 

                                  coef exp(coef)  se(coef)       z Pr(>|z|)    
glucemia_cv_20_22            -0.035001  0.965605  0.001989 -17.598  < 2e-16 ***
HBa1C_cv_20_22                0.046767  1.047877  0.001552  30.130  < 2e-16 ***
MAGE_20_22                          NA        NA  0.000000      NA       NA    
GV_Insulin_Interaction_20_22  0.014421  1.014526  0.003549   4.064 4.83e-05 ***
sexoM                        -0.213778  0.807527  0.002855 -74.875  < 2e-16 ***
pre_dm1                       0.104886  1.110584  0.003100  33.839  < 2e-16 ***
dm2_dxTRUE                          NA        NA  0.000000      NA       NA    
hta_dxTRUE                          NA        NA  0.000000      NA       NA    
insulina_2022                -0.093223  0.910990  0.006725 -13.861  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

                             exp(coef) exp(-coef) lower .95 upper .95
glucemia_cv_20_22               0.9656     1.0356    0.9618    0.9694
HBa1C_cv_20_22                  1.0479     0.9543    1.0447    1.0511
MAGE_20_22                          NA         NA        NA        NA
GV_Insulin_Interaction_20_22    1.0145     0.9857    1.0075    1.0216
sexoM                           0.8075     1.2383    0.8030    0.8121
pre_dm1                         1.1106     0.9004    1.1039    1.1174
dm2_dxTRUE                          NA         NA        NA        NA
hta_dxTRUE                          NA         NA        NA        NA
insulina_2022                   0.9110     1.0977    0.8991    0.9231

Concordance= 0.535  (se = 0 )
Likelihood ratio test= 8231  on 6 df,   p=<2e-16
Wald test            = 8422  on 6 df,   p=<2e-16
Score (logrank) test = 8447  on 6 df,   p=<2e-16



## Proportional Hazards Assumption Test
                              test_statistic             p    -log2(p)
GV_Insulin_Interaction_20_22      321.232296  7.806961e-72  236.214062
HBa1C_cv_20_22                     13.755688  2.081894e-04   12.229816
glucemia_cv_20_22                  16.604673  4.603744e-05   14.406833
insulina_2022                    4067.500096  0.000000e+00         inf
pre_dm_1                           77.515715  1.316706e-18   59.397772
sexo_1                            147.903361  4.980305e-34  110.629321

## Sensitivity Analysis
        covariate      coef  exp(coef)  se(coef)  coef lower 95%  coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%  cmp to          z             p   -log2(p)
glucemia_cv_20_22 -0.013516   0.986574  0.000849       -0.015180       -0.011853             0.984935             0.988217     0.0 -15.924472  4.286152e-57 187.250219
   HBa1C_cv_20_22  0.027098   1.027469  0.000888        0.025359        0.028838             1.025683             1.029258     0.0  30.531076 1.008497e-204 677.661124
    insulina_2022 -0.037867   0.962841  0.003053       -0.043851       -0.031884             0.957097             0.968619     0.0 -12.403793  2.492400e-35 114.949948
           sexo_1 -0.143533   0.866292  0.001873       -0.147204       -0.139862             0.863118             0.869478     0.0 -76.627530  0.000000e+00        inf
         pre_dm_1  0.068884   1.071312  0.001869        0.065220        0.072548             1.067393             1.075244     0.0  36.846332 3.348093e-297 984.869304

## ARIMA Model Summary
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:               530229
Model:                 ARIMA(1, 1, 1)   Log Likelihood             -751267.582
Date:                Sat, 13 Sep 2025   AIC                        1502541.164
Time:                        15:02:33   BIC                        1502574.707
Sample:                             0   HQIC                       1502550.636
                             - 530229                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0011      0.001      0.769      0.442      -0.002       0.004
ma.L1         -0.9933      0.000  -6256.080      0.000      -0.994      -0.993
sigma2         0.9959      0.002    505.321      0.000       0.992       1.000
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):             52435.77
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.96   Skew:                             0.77
Prob(H) (two-sided):                  0.00   Kurtosis:                         2.93
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).

## Collaboration and Clinical Insights
- Continuous collaboration with clinical experts ensured the clinical relevance of our findings.
- Feedback loops with geriatricians and pharmacologists helped interpret results in the context of elderly care.
- The integration of clinical insights was crucial for aligning analytical approaches with clinical priorities.

## Future Directions
- Further exploration of non-linear relationships and interaction terms.
- Consideration of additional temporal analysis techniques.
- Ongoing collaboration with clinical teams to refine and validate findings.


'''

## Discussion 6: Paper drafting

#### 6a. Gather insights from the research team

In [205]:
participants_d6 = {key.title: agent_to_assistant[key.title] for key in [*research_team, coordinator]}

In [206]:
prompt_discussion_6a = Discussion(
    title = "research team’s perspective on the results",
    moderator = coordinator,
    type = "writting draft session",
    goal = " gather group members' insights to support the drafting of a well-structured scientific article",
    contributors = research_team, #aquí (y en individual meetings en general, quizas es más interesante meter directamente el prompt del agente) #recuerda que en este primer prompt se describe al equipo también para asignar las tareas del workflow
    scheme= (1, participants_d6),
    materials = ("electronic health records from 500,000 adults aged over 75 in Madrid. This database originates from a retrospective longitudinal study, covering medical variables recorded during the periods 2015–2019 and 2020–2022.", aged_madrid_dictdb), #quizas aquí ya solo dar logs. results y images_names
    agenda = global_project_agenda + "This is the final workflow followed to address the project objective:\n" + final_workflow + "This is the methodology used to address the main objective\n" + methodology + "\nThese are the results obtained:\n"  + results,
    deliverable= "summary of research_team insights",
    participants_request = "thoughts, comments of relevants details to include in the manuscript or conclussions about the results",
    request  = " please convene the meeting and formulate three distinct questions for each individual group member, aimed at clarifying uncertainties, eliciting insights, and extract conclussions from the results")

In [207]:
print(prompt_discussion_6a.prompt)

This is the beginning of a writting draft session to  gather group members' insights to support the drafting of a well-structured scientific article.
You are working on a research project to evaluate whether GV is an independent predictor of all-cause mortality in patients aged 75 years or older with and without COVID-19 who were followed during the first year of the COVID-19 pandemic and its lockdown importancia using statistical methods.Your research team is composed of the following members:
A Consultant in Geriatric Medicine, expert in clinical interpretation of biochemical metrics, primary care of elderly populations, involvement in aging research projects, and distinguishing intrinsic parametric variation from confounding factors and comorbidities related to aging. Their goal is Providing a geriatric clinical perspective to ensure results are relevant and applicable to older patients. Their role in the project will be Offer expertise on geriatric care and validate the clinical re

In [208]:
comments, log6, thread6 = run_discussion(client, prompt_discussion_6a, coordinator, participants_d6)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_D7u62aVTvTSumc0f0LqZcr7V, content=["Certainly! Let's convene the meeting and proceed with formulating questions for each team member. The goal is to clarify uncertainties, elicit insights, and extract conclusions from the results. \n\n### Consultant in Geriatric Medicine\n1. **Clinical Relevance:** How do you interpret the findings of glycemic variability (GV) as an independent predictor of all-cause mortality in the context of geriatric care? Are there specific age-related factors or comorbidities that should be highlighted in the discussion?\n2. **Practical Recommendations:** Based on the results, what practical recommendations can be made for managing GV in older adults, particularly during the pandemic period?\n3. **Confounding Factors:** How do you suggest we address potential confounding factors related to aging and comorbidities in our analysis and discussion?\n\n### Clinical Pharmacology Specialist\n1. **Medicatio

In [209]:
paper_sections_instructions = {
    "Introduction": '''Consider the prior viewpoints of the discussion group members, where relevant to this section. In the introduction, please include: (a) The general context and relevance of the research; (b) The state of the art; (c) The knowledge gap addressed by the study and (d) The specific objective of the study.''',
    "Materials and methods": '''(a) Any datasets, tools, or software used to support the experiment, including preprocessing or formatting steps if applicable; (b) Measures to ensure reproducibility, reliability, and transparency of the experiment, including reference to a GitHub repository for accessing code and data; (c) Additional methodological details or insights that help the reader fully understand the experimental setup, including potential limitations''',
    "Results": '''(a) Present the key results in clear and organized tables. (b) Comments interpreting the analysis of the results observed in the table, highlighting important aspects and potential implications. (c)If relevant, include descriptions of any images, graphs, or charts that can help visualize the results. The graphs should clearly illustrate the findings, such as the relationship between variable''',
    "Discussion": '''(a) Interpretation of the main findings; (b) Implications of these findings for the objective and the hypothesis of the research.''',
    "Conclussions":'''(a) A concise summary of the main findings from the experiment. (b) Discussion of the strengths and limitations of research; (c) Suggestions for future research or potential ways to address observed limitations; (d) Optional: insights into how the experiment contributes to the knowledge generation.''',
    "Abstract":'''A concise summary of the main points for each section, limited to two-three lines''',
    "References":'''(a) Only scientific articles with valid DOIs that are publicly available before April 2024 on PubMed or Google Scholar; (b) Do not generate or include any DOI numbers that do not correspond to actual, verifiable publications. All DOIs must be real and traceable via PubMed, Google Scholar, or similar trusted academic databases; (c) All statements in introduction and materials and methods sections should be referenced. Software, tools, packages or models also should be referenced; (d) Every reference in references section must be cited at least once in the text of the article (in-line citations should match the references listed). (e) Do not include any reference in the text that is not listed in the References section, and do not list any reference in the References section that is not cited in the text; (f) Ensure that all references follow a consistent, standard academic citation style (e.g., Vancouver or APA, at your discretion); (g) Include references for all sections of the article (Introduction, Materials & Methods, Results, Discussion, Conclusions), ensuring that each citation is appropriately linked to the context in which it is used. The total number of references for the manuscript should exceed 20'''
}

In [210]:
writting_advice = '''
Here are some tips for writing your paper:
(a) Do not itemize or use bullet points; write in well-connected, coherent paragraphs.
(b) Avoid mentioning specific column names or variable names from the database. Instead, refer to the concept or measurement the variable represents.
(c) Focus on explaining the idea, rationale, or finding rather than referencing the raw data structure.
(d) Make transitions smooth between sentences and paragraphs, ensuring logical flow.
(e) Provide clear interpretation, reasoning, and context, suitable for a scientific audience, without exposing technical table or dataset identifiers.
(f) Keep language formal, precise, and informative, as for a methods, results, or discussion section of a paper.
(g) The extension of the complete manuscript should be 8 pages. So please, Write each section in detailed paragraphs, using at least 3-5 paragraphs per section. Each paragraph should be fully developed with clear explanations and logical flow
'''

#### 6b. Writting a manuscript

In [211]:
prompt_discussion_6b = Discussion(
    title = "Writting manuscript",
    moderator = coordinator,
    type = "writting draft part of the session",
    goal = "write clear and well-organized scientific article that explains how the project was carried out, along with the results and conclusions obtained",
    contributors = coordinator,
    agenda = f"These are the instructions to write each section of the paper: {paper_sections_instructions}\n{writting_advice}",
    deliverable= "",
    request  = " begin giving a summary of research team insights before start with the writting sections rounds. Consider the prior viewpoints of the discussion group members, where relevant to each section")

In [212]:
print(prompt_discussion_6b.prompt)

This is the beginning of a writting draft part of the session to write clear and well-organized scientific article that explains how the project was carried out, along with the results and conclusions obtained.
These are the instructions to write each section of the paper: {'Introduction': 'Consider the prior viewpoints of the discussion group members, where relevant to this section. In the introduction, please include: (a) The general context and relevance of the research; (b) The state of the art; (c) The knowledge gap addressed by the study and (d) The specific objective of the study.', 'Materials and methods': '(a) Any datasets, tools, or software used to support the experiment, including preprocessing or formatting steps if applicable; (b) Measures to ensure reproducibility, reliability, and transparency of the experiment, including reference to a GitHub repository for accessing code and data; (c) Additional methodological details or insights that help the reader fully understand 

In [213]:
paper_drafting_log = run_paper_drafting(client, prompt_discussion_6b, coordinator, paper_sections_instructions, participants=participants_d6, thread_id=thread6)

Primary Healthcare Professional & Coordinator: The research team provided valuable insights into the study of glycemic variability (GV) as an independent predictor of all-cause mortality in older adults during the COVID-19 pandemic. The Consultant in Geriatric Medicine emphasized the clinical relevance of GV, highlighting the need to consider age-related factors and comorbidities. Practical recommendations for managing GV, particularly during the pandemic, were suggested, along with the importance of addressing potential confounding factors related to aging.

The Clinical Pharmacology Specialist focused on the impact of polypharmacy and medication-related confounding factors on GV and mortality. The need for careful medication management and the implications for prescribing practices were emphasized, along with understanding the interaction between GV and specific medications, such as insulin.

The Endocrinologist supported the robustness and clinical relevance of GV metrics and discus

In [228]:
manuscript = ""
for section, content, _ in paper_drafting_log:
    if section == "Primary Healthcare Professional & Coordinator":
        continue
    manuscript+= section[1:] +"\n"+content+"\n\n"
print(manuscript)

Introduction
The COVID-19 pandemic has posed unprecedented challenges to healthcare systems worldwide, particularly affecting older adults who are more vulnerable to severe outcomes. In this context, understanding the factors that contribute to mortality in this population is of paramount importance. Glycemic variability (GV), which refers to fluctuations in blood glucose levels, has emerged as a potential predictor of adverse health outcomes. This study focuses on evaluating GV as an independent predictor of all-cause mortality in individuals aged 75 years and older, both with and without COVID-19, during the first year of the pandemic.

Current literature highlights the significance of GV in the management of diabetes, with studies linking high GV to increased risks of cardiovascular events, oxidative stress, and inflammation. These mechanisms are particularly detrimental in older adults, who often have diminished physiological reserves and multiple comorbidities. Despite the growing

## Discussion 7: Peer review process

In [214]:
guideliness_NeuroIPS = '''
You are acting as a reviewer for NeurIPS 2025. Follow these principles:
Review Requirements
(a)Each review must be thoughtful, specific, constructive, fair, and timely. Avoid vague statements or superficial critiques.
(b)Structure of Review
    1. Summary – Concisely explain the paper’s contributions in your own words (not the abstract).
    2. Strengths & Weaknesses – Evaluate along four dimensions:
    3. Quality: Technical soundness, validity of claims, completeness.
    4. Clarity: Writing, organization, reproducibility.
    5. Significance: Impact, usefulness, advancement of knowledge.
    6. Originality: Novelty of insights, methods, or combinations.
    7. Questions for Authors – Provide 3–5 constructive, actionable questions or clarifications.
    8. Limitations – Check if limitations and potential societal impacts are addressed. Reward transparency.
    9. Ratings (1–4 for each dimension; 1–6 overall). Clearly justify scores.
    10. Confidence (1–5). Indicate certainty in your evaluation.
    11. Ethical Concerns – Flag for ethics review if relevant.
(d)Best Practices
- Be open-minded during rebuttal and discussions. Revise your review if authors address concerns.
- Be fair and respectful; avoid biased or discouraging language.
- Focus on substance (technical quality, clarity, impact, novelty) rather than minor formatting or grammar.
- Report plagiarism, dual submissions, or unethical behavior to your AC.
- Do not use ideas, code, or results until they are public.
- Run any submitted code only in a safe, isolated environment.

'''

In [ ]:
latex_template = ""

In [247]:
participants_d7 = {key.title: agent_to_assistant[key.title] for key in [*reviewers, coordinator]}
prompt_discussion_7 = Discussion(
    title = "Peer review process ",
    moderator = coordinator,
    type = "peer review process",
    goal = "gather in-depth feedback from Reviewer 1 and Reviewer 2 to refine and strengthen the manuscript before its final submission to the conference",
    scheme = (2, participants_d7),
    deliverable = "final manuscript embeed in latex code template, ready to the conference submission",
    contributors = reviewers, 
    materials = "",
    agenda = f"This is the manuscript:\n{manuscript}\nThis is the latex template including the format that submissions should follow:\n {latex_template}.\n This is the guide to reviewers:\n {guideliness_NeuroIPS}",
    request  = f" please embeed the manuscript in latex template to start the revision process.",
    participants_request = "provide your observations on the manuscript, focusing on two main aspects: 1) ensuring the manuscript follows the conference LaTeX template, and 2) evaluating the quality, clarity, significance, and originality of the content, in line with the NeurIPS 2025 review guidelines. Your expertise will be invaluable for assessing both the formatting and the scientific contribution of the manuscript",
    final_request = "provide the final manuscript embeed in latex code template, ready to the conference submission."
    )

In [248]:
print(prompt_discussion_7.prompt)

This is the beginning of a peer review process to gather in-depth feedback from Reviewer 1 and Reviewer 2 to refine and strengthen the manuscript before its final submission to the conference.
This is the manuscript:
Introduction
The COVID-19 pandemic has posed unprecedented challenges to healthcare systems worldwide, particularly affecting older adults who are more vulnerable to severe outcomes. In this context, understanding the factors that contribute to mortality in this population is of paramount importance. Glycemic variability (GV), which refers to fluctuations in blood glucose levels, has emerged as a potential predictor of adverse health outcomes. This study focuses on evaluating GV as an independent predictor of all-cause mortality in individuals aged 75 years and older, both with and without COVID-19, during the first year of the pandemic.

Current literature highlights the significance of GV in the management of diabetes, with studies linking high GV to increased risks of c

In [249]:
article, log7, thread = run_discussion(client, prompt_discussion_7, coordinator, participants_d7)

=== Mensajes en el hilo ===
0: role=assistant, assistant_id=asst_D7u62aVTvTSumc0f0LqZcr7V, content=["Certainly! Below is the manuscript embedded in the provided LaTeX template. This will serve as the basis for the peer review process.\n\n```latex\n\\documentclass{article}\n\n\\usepackage{agents4science_2025}\n\\usepackage[utf8]{inputenc}\n\\usepackage[T1]{fontenc}\n\\usepackage{hyperref}\n\\usepackage{url}\n\\usepackage{booktabs}\n\\usepackage{amsfonts}\n\\usepackage{nicefrac}\n\\usepackage{microtype}\n\\usepackage{xcolor}\n\n\\title{Glycemic Variability as an Independent Predictor of All-Cause Mortality in Older Adults During the COVID-19 Pandemic}\n\n\\author{%\n  Primary Healthcare Professional \\\\\n  Department of Primary Care \\\\\n  Health Institute of Madrid \\\\\n  Madrid, Spain \\\\\n  \\texttt{primarycare@healthinstitute.es} \\\\\n}\n\n\\begin{document}\n\n\\maketitle\n\n\\begin{abstract}\n  This study investigates glycemic variability (GV) as an independent predictor of all